In [ ]:
import pickle
import numpy as np
import os
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt

def load_deap_dat(file_path):
    with open(file_path, 'rb') as file:
        subject_data = pickle.load(file, encoding='latin1')
    return subject_data['data'][:, :32, :]

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    return butter(order, [lowcut / nyq, highcut / nyq], btype='band')

def bandpass_filter(data, lowcut=4, highcut=30, fs=128, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return lfilter(b, a, data)

def bilateral_filter(signal_data, epsilon_i=0.1, epsilon_m=0.1, window_size=5):
    half_window = window_size // 2
    filtered = np.zeros_like(signal_data)
    for i in range(len(signal_data)):
        weights = []
        values = []
        for j in range(-half_window, half_window + 1):
            idx = i + j
            if 0 <= idx < len(signal_data):
                t_weight = np.exp(-(j ** 2) / (2 * epsilon_i ** 2))
                a_weight = np.exp(-((signal_data[idx] - signal_data[i]) ** 2) / (2 * epsilon_m ** 2))
                w = t_weight * a_weight
                weights.append(w)
                values.append(w * signal_data[idx])
        filtered[i] = np.sum(values) / (np.sum(weights) + 1e-8)
    return filtered

def DBLSHF(signal_data, epsilon_i=0.3, epsilon_m=0.2, lambd=1.2):
    base_layer = bilateral_filter(signal_data, epsilon_i, epsilon_m)
    detail_layer = signal_data - base_layer
    smoothed_signal = signal_data + lambd * detail_layer
    final_preprocessed = bilateral_filter(smoothed_signal, epsilon_i, epsilon_m)
    
    return final_preprocessed, base_layer, detail_layer, smoothed_signal

def process_trial(eeg_trial, fs=128):
    processed_channels = []
    for ch in eeg_trial:
        filtered = bandpass_filter(ch, fs=fs)
        final_output, _, _, _ = DBLSHF(filtered)
        processed_channels.append(final_output)
    return np.array(processed_channels)

